In [1]:
import pandas as pd
from datetime import date, timedelta


In [79]:

class DataWrangler:
    """
    DataWrangler extracts Covid data from https://healthdata.gov/ and provides the following metrics:
        - The total number of PCR tests performed as of a particular day (total_pcr_date) in the United States.
        - The n-day (window) rolling average number of new cases per day for the last k (rolling_averages_days) days.
        - The top n (top_states) states with the highest test positivity rate (positive tests / tests performed) for tests performed in the last k (positivity_rates_days) days.

    Parameters:
        total_pcr_date: date. Date up until when total pcr tests should calculated to. Default = current date - 1 day.
        window: int. Number of days for the rolling average window. Defaul = 7.
        rolling_averages_days: int. Number of days to be caculated for the rolling average. Default = 30 days.
        positivity_rates_days: date. Date of the earliest day to start calculating positivity rate. Default = current date - 30 days.
        top_states: int. Number of top states with highest positivity rates.
    """

    def __init__(self,
            total_pcr_date=(date.today()-timedelta(days=1)).strftime('%Y-%m-%d'),
            window=7,
            rolling_averages_days=30,
            positivity_rates_days=30,
            # (date.today()-timedelta(days=30)).strftime('%Y-%m-%d'),
            top_states=10):
        
        url = 'https://healthdata.gov/resource/j8mb-icvb.json?$limit=300000'

        self.covid_data = pd.read_json(url)
        self.total_pcr_date = total_pcr_date
        self.window = window
        self.rolling_averages_days = rolling_averages_days
        self.positivity_rates_days = positivity_rates_days
        self.positivity_rates_date = (date.today()-timedelta(days=positivity_rates_days)).strftime('%Y-%m-%d')
        self.top_states = top_states

    def total_pcr_test(self):
        """
        Calculates the total number PCR tests done as at a particular day
        :return: total number of PCR test as at a given day
        """
        return self.covid_data[self.covid_data['date']<=self.total_pcr_date]['total_results_reported'].sum()

    def rolling_average(self):
        """
        Calulates the n-day rolling average of new positive cases in the last k days
        :return: rolling averages
        """
        # new cases includes only positive cases
        new_cases_sum = self.covid_data[self.covid_data['overall_outcome']=='Positive'].groupby(['date'])['new_results_reported'].sum()
        new_cases_sum.sort_index()
        
        windows = new_cases_sum.rolling(self.window)
        rolling_averages = windows.mean()
        return rolling_averages[-self.rolling_averages_days:]

    def top_n_states(self):
        """
        Calculates positivity rate by states. First gets total tests and total positive tests then divide
        :return: top n states with highest positivity rates
        """
        total_test_df = pd.DataFrame(self.covid_data[self.covid_data['date']>=self.positivity_rates_date]\
            .groupby(['state_name'])['total_results_reported'].sum())

        positive_test_df = pd.DataFrame(self.covid_data[(self.covid_data['overall_outcome']=='Positive') \
            & (self.covid_data['date']>=self.positivity_rates_date)].groupby(['state_name'])['total_results_reported'].sum())

        positive_test_df.rename(columns={'total_results_reported': 'positive_results'}, inplace=True)

        joined_df = total_test_df.join(positive_test_df)
        joined_df['positivity_rate'] = round(joined_df['positive_results'] / joined_df['total_results_reported'],3)
        
        return joined_df[['positivity_rate']].sort_values(by=['positivity_rate'], ascending=False).head(self.top_states)

    def main(self):
        print(f'\n1. The total number of PCR tests performed as at {self.total_pcr_date} in the United State is {self.total_pcr_test()}\n')

        print(f'2. The {self.window}-day rolling average number of new cases per day for the last {self.rolling_averages_days} days is \n{self.rolling_average()}\n')

        print(f'3. The {self.top_states} states with the highest test positivity rate (positive tests / tests performed) for tests performed in the last {self.positivity_rates_days} days is {self.top_n_states()}')
        

In [80]:
obj = DataWrangler()
# obj.total_pcr_test()
# obj.rolling_average()
# obj.top_n_states()
obj.main()


1. The total number of PCR tests performed as at 2022-05-22 in the United State is 317858748732

2. The 7-day rolling average number of new cases per day for the last 30 days is 
date
2022-04-21    34272.857143
2022-04-22    34821.714286
2022-04-23    35282.714286
2022-04-24    36237.428571
2022-04-25    39042.142857
2022-04-26    42950.142857
2022-04-27    46806.428571
2022-04-28    50961.000000
2022-04-29    54455.428571
2022-04-30    56555.428571
2022-05-01    58098.142857
2022-05-02    59388.142857
2022-05-03    60494.714286
2022-05-04    61724.000000
2022-05-05    62467.428571
2022-05-06    63764.428571
2022-05-07    65641.285714
2022-05-08    67213.714286
2022-05-09    69455.714286
2022-05-10    72624.714286
2022-05-11    76082.285714
2022-05-12    78761.857143
2022-05-13    80478.000000
2022-05-14    82242.000000
2022-05-15    83468.285714
2022-05-16    85481.714286
2022-05-17    86511.571429
2022-05-18    85338.142857
2022-05-19    81489.714286
2022-05-20    73715.714286
Name:

In [28]:
url = 'https://healthdata.gov/resource/j8mb-icvb.json?state=WA'

covid_data = pd.read_json(url)

In [29]:
covid_data

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,WA,Washington,53,Region 10,Inconclusive,2020-03-01,1,1
1,WA,Washington,53,Region 10,Negative,2020-03-01,26,26
2,WA,Washington,53,Region 10,Positive,2020-03-01,28,28
3,WA,Washington,53,Region 10,Inconclusive,2020-03-02,0,1
4,WA,Washington,53,Region 10,Negative,2020-03-02,37,63
...,...,...,...,...,...,...,...,...
995,WA,Washington,53,Region 10,Positive,2021-01-26,972,214552
996,WA,Washington,53,Region 10,Inconclusive,2021-01-27,3,1108
997,WA,Washington,53,Region 10,Negative,2021-01-27,13489,3215455
998,WA,Washington,53,Region 10,Positive,2021-01-27,990,215542


In [22]:
data = covid_data.groupby(['date'])['new_results_reported'].sum()
window = 7

In [23]:
type(data)

pandas.core.series.Series

In [26]:
data.sort_index()

date
2020-03-01       55
2020-03-02       55
2020-03-03       49
2020-03-04      169
2020-03-05       98
              ...  
2021-01-24     9063
2021-01-25     7450
2021-01-26    16537
2021-01-27    14482
2021-01-28        4
Name: new_results_reported, Length: 334, dtype: int64

In [15]:
windows = data.rolling(window)

In [16]:
rolling_averages = windows.mean()

In [17]:
rolling_averages[-30:]

date
2020-12-30    13659.000000
2020-12-31    13451.142857
2021-01-01    14310.000000
2021-01-02    14655.285714
2021-01-03    14820.285714
2021-01-04    15121.428571
2021-01-05    14838.000000
2021-01-06    14928.285714
2021-01-07    15163.428571
2021-01-08    15856.428571
2021-01-09    17215.571429
2021-01-10    17181.285714
2021-01-11    17049.714286
2021-01-12    16972.714286
2021-01-13    16836.857143
2021-01-14    16633.857143
2021-01-15    15831.142857
2021-01-16    15449.000000
2021-01-17    15039.714286
2021-01-18    14828.857143
2021-01-19    14399.857143
2021-01-20    13745.857143
2021-01-21    13528.000000
2021-01-22    13533.142857
2021-01-23    13062.428571
2021-01-24    12841.571429
2021-01-25    12830.714286
2021-01-26    13194.428571
2021-01-27    12898.000000
2021-01-28    10491.714286
Name: new_results_reported, dtype: float64

In [223]:
covid_data

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,AL,Alabama,1,Region 4,Negative,2020-03-01,96,96
1,AL,Alabama,1,Region 4,Positive,2020-03-01,16,16
2,AL,Alabama,1,Region 4,Negative,2020-03-02,72,168
3,AL,Alabama,1,Region 4,Positive,2020-03-02,6,22
4,AL,Alabama,1,Region 4,Negative,2020-03-03,94,262
...,...,...,...,...,...,...,...,...
995,AL,Alabama,1,Region 4,Positive,2021-01-30,1383,481176
996,AL,Alabama,1,Region 4,Inconclusive,2021-01-31,8,8693
997,AL,Alabama,1,Region 4,Negative,2021-01-31,4614,3236626
998,AL,Alabama,1,Region 4,Positive,2021-01-31,986,482162


In [209]:
total_test_df = pd.DataFrame(covid_data[covid_data['date']>='2021-11-01'].groupby(['state'])['total_results_reported'].sum())
total_test_df

,total_results_reported
state,
AK,4148772
AL,7624753
AR,4731910
AZ,14802197
CA,137294688
CO,15861051
CT,14573051
DC,5021568
DE,3677601


In [210]:
positive_test_df = pd.DataFrame(covid_data[(covid_data['overall_outcome']=='Positive') & (covid_data['date']>='2022-01-01')].groupby(['state'])['total_results_reported'].sum())
positive_test_df.rename(columns={'total_results_reported': 'positive_results'}, inplace=True)
positive_test_df.fillna(0)

,positive_results
state,
AK,309900
AL,1076660
AR,496391
AZ,1958348
CA,8692200
CO,1451888
CT,880531
DC,221404
DE,290583


In [203]:
positive_test_df = pd.DataFrame(covid_data[(covid_data['overall_outcome']=='Positive') & (covid_data['date']>='2022-01-01')])
positive_test_df

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported


In [206]:
total_test_df = pd.DataFrame(covid_data[covid_data['date']>='2021-01-01'])
total_test_df

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
906,AL,Alabama,1,Region 4,Inconclusive,2021-01-01,86,7644
907,AL,Alabama,1,Region 4,Negative,2021-01-01,10768,2874405
908,AL,Alabama,1,Region 4,Positive,2021-01-01,2548,407644
909,AL,Alabama,1,Region 4,Inconclusive,2021-01-02,42,7686
910,AL,Alabama,1,Region 4,Negative,2021-01-02,6113,2880518
...,...,...,...,...,...,...,...,...
995,AL,Alabama,1,Region 4,Positive,2021-01-30,1383,481176
996,AL,Alabama,1,Region 4,Inconclusive,2021-01-31,8,8693
997,AL,Alabama,1,Region 4,Negative,2021-01-31,4614,3236626
998,AL,Alabama,1,Region 4,Positive,2021-01-31,986,482162


In [207]:
covid_data

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,AL,Alabama,1,Region 4,Negative,2020-03-01,96,96
1,AL,Alabama,1,Region 4,Positive,2020-03-01,16,16
2,AL,Alabama,1,Region 4,Negative,2020-03-02,72,168
3,AL,Alabama,1,Region 4,Positive,2020-03-02,6,22
4,AL,Alabama,1,Region 4,Negative,2020-03-03,94,262
...,...,...,...,...,...,...,...,...
995,AL,Alabama,1,Region 4,Positive,2021-01-30,1383,481176
996,AL,Alabama,1,Region 4,Inconclusive,2021-01-31,8,8693
997,AL,Alabama,1,Region 4,Negative,2021-01-31,4614,3236626
998,AL,Alabama,1,Region 4,Positive,2021-01-31,986,482162


In [126]:
joined_df = total_test_df.join(positive_test_df)
joined_df['positivity_rate'] = joined_df['positive_results'] / joined_df['total_results_reported']
joined_df.sort_values(by=['positivity_rate']).head(10)

,total_results_reported,positive_results,positivity_rate
state,,,
AL,481890931,57282289,0.11887


In [49]:
dr = 'https://soda.demo.socrata.com/resource/4tka-6guv.json?$where=magnitude>3.0'
drt = pd.read_json(dr)
drt

,source,earthquake_id,magnitude,depth,number_of_stations,region,location,version
0,ci,ci37219008,3.04,2.7200,3.0,"51km NNE of Camalu, Mexico",NaN,NaN
1,ci,ci15498737,3.04,0.0300,4.0,"47km NNE of Camalu, Mexico",NaN,NaN
2,nn,nn00445547,3.06,1.8345,12.0,"16km SW of Hawthorne, Nevada",NaN,NaN
3,ci,ci37218200,3.07,5.1900,3.0,"33km N of Camalu, Mexico",NaN,NaN
4,ci,ci15499489,3.09,19.1600,113.0,"5km E of Cabazon, California",NaN,NaN
...,...,...,...,...,...,...,...,...
995,us,usc000r8gs,5.30,35.0000,NaN,"107km S of Raoul Island, New Zealand","{'latitude': '-30.2394', 'longitude': '-177.86...",NaN
996,us,usb000qu45,5.30,10.0000,NaN,Central East Pacific Rise,"{'latitude': '-4.385', 'longitude': '-105.8958'}",NaN
997,us,usb000q6by,5.40,6.2700,NaN,Palau region,NaN,NaN
998,us,c000cng3,5.40,9.8000,94.0,"south of Java, Indonesia","{'latitude': '-10.8088', 'longitude': '113.7383'}",9


In [ ]:
url = 'https://healthdata.gov/resource/j8mb-icvb.json?$where=date<2022-01-01'

covid_data = pd.read_json(url)
covid_data

In [35]:
type(covid_data)

pandas.core.frame.DataFrame

In [9]:
covid_data['total_results_reported'].sum()

481890931

In [16]:
covid_data[covid_data['date']<='2020-11-01']['total_results_reported'].sum()

206252099

In [18]:
covid_data[covid_data['date']<=time.today()]['total_results_reported'].sum()

AttributeError: module 'time' has no attribute 'today'

In [8]:
covid_data

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,AL,Alabama,1,Region 4,Negative,2020-03-01,96,96
1,AL,Alabama,1,Region 4,Positive,2020-03-01,16,16
2,AL,Alabama,1,Region 4,Negative,2020-03-02,72,168
3,AL,Alabama,1,Region 4,Positive,2020-03-02,6,22
4,AL,Alabama,1,Region 4,Negative,2020-03-03,94,262
...,...,...,...,...,...,...,...,...
995,AL,Alabama,1,Region 4,Positive,2021-01-30,1383,481176
996,AL,Alabama,1,Region 4,Inconclusive,2021-01-31,8,8693
997,AL,Alabama,1,Region 4,Negative,2021-01-31,4614,3236626
998,AL,Alabama,1,Region 4,Positive,2021-01-31,986,482162


In [42]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("healthdata.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(healthdata.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("j8mb-icvb", limit=131302)

# Convert to pandas DataFrame
# results_df = pd.read_json(results)
results_df = pd.DataFrame.from_records(results)

In [148]:
type(results)

list

In [93]:
results_df.groupby('state')['total_results_reported'].sum()

state
AK    4461827405058779013922162356342435383746810641...
AL    9616168222623126233323393354335547449645077059...
AR    1222566638318624672886101102818647217514445810...
AZ    1116112111311358165816591672189725143251792524...
CA    4512143282193828144445632625883106713631199180...
CO    1924578414017621423312461262832812721008162101...
CT    2471531455725729641296167812051612482512994513...
DC    3317241371625132923616286203482342834710639558...
DE    2222446815245611534714131880111180161360211696...
FL    3111762026341226154228305921010652114879214481...
GA    1317122144192511561581133720596319237515351401...
GU    4162226456486101181814229178332223526459335634...
HI    1555688111116192611443243641666421011012011772...
IA    1111112444558191101291401511532584655655825906...
ID    1228456912811421162519751216512525126851331513...
IL    4831554119551354814149147813154122170541184856...
IN    1716536083698614948176601859625110930510941313...
KS    565659626669732762289221082212552142